In [ ]:
# Step 1: Install required packages
!pip install flask pyngrok requests
!pip install stanza
!pip install nlp
!pip install pandas
!pip install ast
!pip install googletrans==4.0.0-rc1
!pip install pyiwn
!pip install flask-cors


# Step 2: Import required modules
import stanza
import nlp
import pandas as pd
import ast
import re
import os
import subprocess
import time
import pyiwn
from flask import Flask, request, render_template_string  # Fix: Added render_template_string
from flask_cors import CORS  # <--- NEW
from pyngrok import ngrok
from nltk.corpus import wordnet as wn
import requests
from moviepy.editor import VideoFileClip, concatenate_videoclips
from IPython.display import display, HTML
import nltk
import spacy
from googletrans import Translator

nltk.download('wordnet')
os.makedirs("/content/static", exist_ok=True)

# Import Stanza Hindi Pipeline
nlp = stanza.Pipeline('hi', processors='tokenize,lemma,pos,depparse')
# Known POS Tags
print(nlp.processors['pos'].get_known_xpos())

# Step 3: Create a Flask application
app = Flask(__name__, static_folder="/content/static")
CORS(app)

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        sentence = request.form.get("sentence", "Guest")  # Get Hindi sentence

        print(sentence)

        # Main Code Satrts from here
        doc = nlp(sentence)
        # Extract relevant fields from the data
        df_data = [(token['text'], token['upos'], token['xpos'], token['head'], token['deprel']) for token in doc.to_dict()[0]]

        # Create DataFrame
        df = pd.DataFrame(df_data, columns=['text', 'upos', 'xpos', 'head', 'deprel'])
        df

        # Extract tags for hindi sentences
        word_tag = []
        for sent in doc.sentences:
            for word in sent.words:
                word_dict = {
                    "text": word.text,
                    "xpos": word.xpos,
                    "id": word.id,
                    "lemma": word.lemma,
                    "deprel": word.deprel,
                    "head": word.head
                }
                word_tag.append(word_dict)

        # Removing Unwanted Tags - VAUX CC SYM
        unwanted_tags = ['VAUX','CC','SYM','PSP']
        word_tag_cleaned = {}
        for rel_dict in word_tag:
            if rel_dict['xpos'] not in unwanted_tags:
                word_tag_cleaned[rel_dict['id']] =  rel_dict
        #print(word_tag_cleaned)

        for key,value in word_tag_cleaned.items():
            print(key,value)

        def change_dict_order(sample_dict,row_1,row_2):
            if row_1 == row_2:
                return sample_dict
            list_form = [(key,value) for key,value in sample_dict.items()]
            index_1 = None     # index_1
            index_2 = None
            for i,content in enumerate(list_form):
                if content[0] == row_1:
                    index_1 = i
                elif content[0] == row_2:
                    index_2 = i
            entry_1 = list_form.pop(index_1)
            list_form.insert(index_2,entry_1)
            return_dict = {}
            for key,value in list_form:
                return_dict[key] = value
            return return_dict

        word_sign_form = word_tag_cleaned.copy()
        # Format Subject then Object
        subject_id = None
        object_id = None
        subject_index = None
        object_index = None
        cnt = 0
        for key,value in word_tag_cleaned.items():
            if value['deprel'] in ['obj','obl']:
                object_id = key
                object_index = cnt
            elif value['deprel']=='nsubj':
                subject_id = key
                subject_index = cnt
            cnt+=1
        #print(subject_id,object_id)
        if subject_id!=None and object_id!=None and subject_id>object_id:
            word_sign_form = change_dict_order(word_sign_form,object_id,subject_id)

        for key,value in word_sign_form.items():
            print(key,value)

        # Arrange Adjective and Adverb
        for key,value in word_tag_cleaned.items():    # word_tag_cleaned
            if value['xpos'] in ['JJ']:     # Adjective Adverb
                # first condition is for when it does not have corresponding noun or verb - मैं खुश हूं।
                if value['head']!=0 and word_tag_cleaned[value['head']]['xpos'] in ['NN']:
                    word_sign_form = change_dict_order(word_sign_form,key,value['head'])
            elif value['xpos'] in ['RB']:
                if value['head']!=0 and word_tag_cleaned[value['head']]['xpos'] in ['VM','VAUX']:
                    word_sign_form = change_dict_order(word_sign_form,key,value['head'])
        #print(word_sign_form)

        # Arrange Negative Sentences
        for key,value in word_sign_form.items():
            if value['xpos']=='NEG':
                last_key = list(word_sign_form.keys())[-1]
                word_sign_form = change_dict_order(word_sign_form,key,last_key)

        # Handling Interrogative Sentence
        for key,value in word_sign_form.items():
            if value['xpos']=='WQ':
                last_key = list(word_sign_form.keys())[-1]
                word_sign_form = change_dict_order(word_sign_form,key,last_key)

        # Read stopwords from file
        with open('/content/drive/MyDrive/final_stopwords.txt', 'r',encoding='utf8') as file:
            # Read the entire contents of the file
            stopword_list = file.readlines()
        stopword_list = [word.strip() for word in stopword_list]

        # StopWord Removal
        stopword_removed_list = {}
        for key,value in word_sign_form.items():
            #print(value['text'] in stopword_list)
            if value['text'] not in stopword_list:
                stopword_removed_list[key] = value

        for key,value in stopword_removed_list.items():
            print(key,value)

        # Mapping xpos to POS tags
        xpos_to_pos = {
            'NNP': 'pnoun',
            'VM': 'verb',
            'VAUX': 'verb',
            'JJ': 'adjective',
            'RB': 'adverb',
            'PRP' : 'pronoun',
            'NEG' : 'negative',
            'NN' : 'noun',
            'RDP' : 'adverb',
            'QF': 'adjective',            # 'अधिक'\
            'WQ': 'wh_adverb',
            'NST': 'noun_locative',
            'DEM': 'noun_refer_specific',
            'INTF': 'intensifier',
            # Add more mappings as needed
        }

        # Extract Words from Parser and corresponding tag
        sign_words_list = []
        for key,value in stopword_removed_list.items():
            if value['xpos'] in xpos_to_pos:
                sign_words_list.append((value['text'], xpos_to_pos[value['xpos']]))
            else:
                sign_words_list.append((value['text'], 'extra'))
        print(sign_words_list)

        # READ ISL Dictionary
        # Open the file in read mode
        with open('/content/drive/MyDrive/isl_dict.txt', 'r',encoding='utf8') as file:
            # Read the entire contents of the file
            isl_dict = ast.literal_eval(file.read())

        ## Use Case : Why_(Sign_2) should be why
        # Remove content within parentheses and strip whitespace for keys containing "_(*)"
        isl_dict = {re.sub(r'_\(.*\)', '', key).strip().lower(): value for key, value in isl_dict.items()}

        # Create a new dictionary with lowercase keys
        isl_dict = {key.lower(): value for key, value in isl_dict.items()}
        isl_dict['school'] = 'स्कूल'

        iwn = pyiwn.IndoWordNet()
        print(dir(iwn))
        print(iwn.synsets('आम्र'))
        print(iwn.all_synsets)

        # Google Translator
        # Create a Translator object
        translator = Translator()

        # Handling Special Case for Mapping to Videos
        special_videos = {
            #'i' : '/content/drive/MyDrive/ISL Dictionary/I/I_Me.mp4',
            'i' : 'I',
            'who': '/content/drive/MyDrive/ISL Dictionary/W/Who_Whom.mp4',
            'whom': '/content/drive/MyDrive/ISL Dictionary/W/Who_Whom.mp4',
            #'happy':'/content/drive/MyDrive/ISL Dictionary/H/Happy.mp4'
        }

        lemmatizer = spacy.load("en_core_web_sm")

        synonym_substituted_list = []
        temp_list = [('आकलन', 'noun')]
        for word,pos_tag in sign_words_list:

            # Translate the Hindi sentence to English
            english_word = translator.translate(word, src='hi', dest='en').text.lower()

            english_word_lemmatized = lemmatizer(english_word)[0].lemma_.lower()

            if english_word_lemmatized in list(special_videos.keys()):
                synonym_substituted_list.append((word,pos_tag,special_videos[english_word_lemmatized]))
                continue

            if pos_tag == 'pnoun':
                synonym_substituted_list.append((word,pos_tag,english_word))
                continue
            #print(word,pos_tag)

            # Case 1 : Check hindi word in isl_dict
            if word in list(isl_dict.values()):
                synonym_substituted_list.append((word,pos_tag,english_word))
                continue
            all_hindi_synsets = []
            # Case 2 : Check synonym of hindi_word in isl_dict
            try:
                all_hindi_synsets = iwn.synsets(word)
            except Exception as e:
                pass
            flag = False
            for synset in all_hindi_synsets:
                if synset._head_word in list(isl_dict.values()):
                    corresponding_keys = [key for key, value in isl_dict.items() if value == synset._head_word]
                    synonym_substituted_list.append((word,pos_tag,corresponding_keys[0]))
                    flag = True
                    break
            if flag == True:
                continue

            # Case 3 : Check english word in isl_dict
            if english_word in list(isl_dict.keys()):
                synonym_substituted_list.append((word,pos_tag,english_word))
                continue

            # Case 4 : Check lemmatized english word in isl_dict
            if english_word_lemmatized in list(isl_dict.keys()):
                synonym_substituted_list.append((word,pos_tag,english_word_lemmatized))
                continue

            # Case 5 : Check syno of english word in isl_dict
            all_english_synsets = wn.synonyms(english_word)
            #print(all_english_synsets)
            all_english_synsets_flatten = []
            for row in all_english_synsets:
                all_english_synsets_flatten.extend(row)
            flag = False
            for synset in all_english_synsets_flatten:
                if synset.lower() in list(isl_dict.keys()):
                    flag = True
                    # print('Yes Present')
                    synonym_substituted_list.append((word,pos_tag,synset.lower()))
                    break
            if flag == True:
                continue

            # Case 6  : Nothing Words Go for Finger Spelling
            synonym_substituted_list.append((word,pos_tag,'#'))
        print(synonym_substituted_list)

        # Final ISL List
        final_isl_list = synonym_substituted_list.copy()
        # Create DataFrame
        df = pd.DataFrame(final_isl_list, columns=['Hindi Word', 'POS Tag', 'ISL Dictionary Tag'])
        df

        # Reversed dictionary mapping Hindi words to English words
        isl_hindi_english_dict = {hindi_word: english_word for english_word, hindi_word in isl_dict.items()}
        print(isl_hindi_english_dict)

        final_isl_list

        # Mapping of Devanagari vowel signs to their vowels
        sign_mapping_vowels = {
            'ा': 'आ',  # Aa
            'ि': 'इ',  # I
            'ी': 'ई',  # II
            'ु': 'उ',  # U
            'ू': 'ऊ',  # UU
            'ृ': 'ऋ',  # R
            'े': 'ए',  # E
            'ै': 'ऐ',  # AI
            'ो': 'ओ',  # O
            'ौ': 'औ',  # AU
            'ं': 'अं', # Anusvara
            'ः': 'अः'  # Visarga
        }

        def search_videos(folder_path, final_isl_list):
            """
            Searches for video files named after the provided words in a directory,
            selecting the best match based on the longest prefix and returning the first
            alphabetically matching video if multiple options exist.
            """
            found_videos = []

            for or_word, pos_tag, isl_word in final_isl_list:
                print(isl_word)
                video_name = f"{isl_word.capitalize()}.mp4"
                best_match = None
                best_match_length = 0
                best_alphabetical_match = None

                if isl_word == 'I':
                    found_videos.append("/content/drive/MyDrive/ISL Dictionary/I/I_Me.mp4")
                    continue

                elif isl_word == '#':
                    for char in or_word:
                        video_name = f"{sign_mapping_vowels[char]}.mp4" if char in sign_mapping_vowels else f"{char}.mp4"
                        for root, dirs, files in os.walk(folder_path):
                            for file in sorted(files):  # Ensuring alphabetical order
                                if file.lower().startswith(video_name.lower().rstrip('.mp4')) and file.endswith('.mp4'):
                                    match_length = len(os.path.splitext(file)[0])
                                    if match_length > best_match_length:
                                        best_match = os.path.join(root, file)
                                        best_match_length = match_length
                                    if best_alphabetical_match is None:
                                        best_alphabetical_match = os.path.join(root, file)
                        if best_match:
                            found_videos.append(best_match)
                        elif best_alphabetical_match:
                            found_videos.append(best_alphabetical_match)
                    continue

                elif isl_word[0] == '@':  # Special Words
                    found_videos.append(isl_word[1:])
                    continue

                else:
                    flag_w = False
                    for root, dirs, files in os.walk(folder_path):
                      if(flag_w==True):
                        break
                      for file in sorted(files):  # Ensuring alphabetical order
                          if file.lower().startswith(isl_word.lower()) and file.endswith('.mp4'):
                              match_length = len(os.path.splitext(file)[0])
                              best_alphabetical_match = os.path.join(root, file)
                              #print("meow")
                              if os.path.exists(best_alphabetical_match):
                                #print("meow")
                                found_videos.append(best_alphabetical_match)
                                flag_w = True
                                break

            return found_videos

        def play_videos(video_paths):
            vlc_path = r'/Applications/VLC.app/Contents/MacOS/VLC'
            for video_path in video_paths:
                subprocess.Popen([vlc_path, '--fullscreen', video_path])
                time.sleep(5)  # Delay before playing the next video

        # Set it accordingly
        # folder_path = 'D:\desktop\project\Linguistic'
        folder_path = '/content/drive/MyDrive/ISL Dictionary'

        video_paths = search_videos(folder_path, final_isl_list)

        video_paths

        #vlc_path = r'C:\Program Files\VideoLAN\VLC\vlc.exe'

        def merge_videos(video_paths):
            clips = [VideoFileClip(path) for path in video_paths]
            final_clip = concatenate_videoclips(clips, method="compose")
            return final_clip

        # Merge the videos into a single video
        merged_clip = merge_videos(video_paths)


        # Export the merged video to a file
        merged_clip.write_videofile("/content/static/merged_video.mp4")

        # Play the merged video
        # subprocess.Popen([vlc_path, '--fullscreen', 'merged_video.mp4'])
        output_path = r'/content/static/merged_video.mp4'

        file_path = "/content/static/merged_video.mp4"
        print("File exists:", os.path.exists(file_path))

        return render_template_string("""
          <h2>आपका वाक्य: {{ sentence }}</h2>
          <video width="640" height="360" controls>
              <source src="{{ url_for('static', filename='merged_video.mp4') }}" type="video/mp4">
          </video>
          <br><a href='/'>वापस जाएं</a>
          """, sentence=sentence)

    return """
    <form action="/" method="POST">
        <label>अपना वाक्य लिखें:</label>
        <input type="text" name="sentence" required>
        <button type="submit">सबमिट करें</button>
    </form>
    """

# 🔑 Set NGROK Auth Token (replace with your actual token)
ngrok.set_auth_token("2xWrUZXu9l5UA4reEvhQMAY1KHb_7t7V4tsSiQUnS3ZX5TSD5")

# Add ngrok tunnel
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{5000}\"")
app.config["BASE_URL"] = public_url

# Step 4: Start the Flask server & expose using Ngrok
# public_url = ngrok.connect(5000)  # Expose Flask app online
# print(f"Public URL: {public_url}")  # Print the public URL

app.run(port=5000)

  Using cached AST-0.0.2.tar.gz (19 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: hi (Hindi):
| Processor | Package       |
-----------------------------
| tokenize  | hdtb          |
| pos       | hdtb_charlm   |
| lemma     | hdtb_nocharlm |
| depparse  | hdtb_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


['PSP', 'NN', 'VM', 'NNP', 'SYM', 'VAUX', 'JJ', 'NNPC', 'PRP', 'CC', 'NNC', 'QC', 'NST', 'DEM', 'RP', 'QF', 'NEG', 'RB', 'QCC', 'QO', 'INTF', 'JJC', 'WQ', 'RDP', 'UNK', 'PRPC', 'NSTC', 'RBC', 'QFC', 'CCC', 'INJ']
 * ngrok tunnel "https://c4d9-34-86-57-165.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:06:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:06:02] "GET /favicon.ico HTTP/1.1" 404 -


...

INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:44:53] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:44:56] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:44:58] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:45:00] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:45:06] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:45:08] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:45:45] "GET / HTTP/1.1" 200 -


आपकी तबीयत कैसी है?
1 {'text': 'आपकी', 'xpos': 'PRP', 'id': 1, 'lemma': 'आप', 'deprel': 'nmod', 'head': 2}
2 {'text': 'तबीयत', 'xpos': 'NN', 'id': 2, 'lemma': 'तबीयत', 'deprel': 'root', 'head': 0}
3 {'text': 'कैसी', 'xpos': 'WQ', 'id': 3, 'lemma': 'कैसा', 'deprel': 'obl', 'head': 2}
4 {'text': 'है', 'xpos': 'VM', 'id': 4, 'lemma': 'है', 'deprel': 'cop', 'head': 2}
1 {'text': 'आपकी', 'xpos': 'PRP', 'id': 1, 'lemma': 'आप', 'deprel': 'nmod', 'head': 2}
2 {'text': 'तबीयत', 'xpos': 'NN', 'id': 2, 'lemma': 'तबीयत', 'deprel': 'root', 'head': 0}
3 {'text': 'कैसी', 'xpos': 'WQ', 'id': 3, 'lemma': 'कैसा', 'deprel': 'obl', 'head': 2}
4 {'text': 'है', 'xpos': 'VM', 'id': 4, 'lemma': 'है', 'deprel': 'cop', 'head': 2}
1 {'text': 'आपकी', 'xpos': 'PRP', 'id': 1, 'lemma': 'आप', 'deprel': 'nmod', 'head': 2}
2 {'text': 'तबीयत', 'xpos': 'NN', 'id': 2, 'lemma': 'तबीयत', 'deprel': 'root', 'head': 0}
3 {'text': 'कैसी', 'xpos': 'WQ', 'id': 3, 'lemma': 'कैसा', 'deprel': 'obl', 'head': 2}
[('आपकी', 'pronoun'), 

INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:48:05] "POST / HTTP/1.1" 200 -


Moviepy - Done !
Moviepy - video ready /content/static/merged_video.mp4
File exists: True


INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:48:07] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:48:09] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:48:11] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:48:12] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [06/Jun/2025 04:48:14] "GET /static/merged_video.mp4 HTTP/1.1" 206 -
